In [2]:
# Imports
from uuid import uuid4
from os import remove
from os.path import join
from tempfile import gettempdir
import numpy as np

import pandas as pd
from aequilibrae.project.database_connection import database_connection
from aequilibrae.utils.db_utils import read_and_close

from aequilibrae.transit import Transit
from aequilibrae.utils.create_example import create_example


# TODO:
# 1. Update preload builder to use pce from database
# 2. Build PC, understand how venv's and wsl system works from ground up, ubuntu 22.04, 
# 3. Remove this notebook from git.

In [3]:
def build_pt_preload(
    self, graph, start: int, end: int, inclusion_cond: str = "start"
) -> np.ndarray:

    # Get trip_id based on specified inclusion condition
    with read_and_close(database_connection("transit")) as conn:
        links = pd.read_sql(build_pt_preload_sql(start, end, inclusion_cond), conn)

    # Calculate non-zero preloads for each link
    links = links.groupby(['link_id', 'direction'], as_index=False)['pce'].sum().rename(columns={"pce": "preload"})

    # Merge preload onto all links/dir's in network and fill in 0 for links with no transit
    preload = pd.merge(graph.graph, links, on=["link_id", "direction"], how="left")
    preload["preload"] = preload["preload"].fillna(0)

    # Extract preload sorted by __supernet_id__ (same ordering as used by capacity in assignment)
    return preload.sort_values(by="__supernet_id__")["preload"].to_numpy()


probe_point_lookup = {
    "start": "MIN(departure)",
    "end": "MAX(arrival)",
    "midpoint": "(MIN(departure) + MAX(arrival)) / 2",
}

def build_pt_preload_sql(start, end, inclusion_cond):

    def select_trip_ids():
        in_period = f"BETWEEN {start} AND {end}"
        if inclusion_cond == "any":
            return f"SELECT DISTINCT trip_id FROM trips_schedule WHERE arrival {in_period} OR departure {in_period}"
        return f"""
            SELECT trip_id FROM trips_schedule GROUP BY trip_id
            HAVING {probe_point_lookup[inclusion_cond]} {in_period}
        """

    # Convert trip_id's to link/dir's via pattern_id's
    return f"""
        SELECT pm.link as link_id, pm.dir as direction, SUM(r.pce) as preload
        FROM (SELECT pattern_id FROM trips WHERE trip_id IN ({select_trip_ids()})) as p 
        INNER JOIN pattern_mapping pm ON p.pattern_id = pm.pattern_id
        INNER JOIN routes r ON p.pattern_id = r.pattern_id
        GROUP BY pm.link, pm.dir
    """

In [4]:
# Build temporary updated coquimbo file

fldr = join(gettempdir(), uuid4().hex)
print(fldr)
project = create_example(fldr, "coquimbo")

remove(join(fldr, "public_transport.sqlite"))
dest_path = join(fldr, "gtfs_coquimbo.zip")

data = Transit(project)

transit = data.new_gtfs_builder(agency="Lisanco", file_path=dest_path)

transit.load_date("2016-04-13")

# Now we execute the map matching to find the real paths.
# Depending on the GTFS size, this process can be really time-consuming.
transit.set_allow_map_match(True)
transit.map_match()

# Finally, we save our GTFS into our model.
transit.save_to_disk()

/tmp/e820432c5b4a45079d3635bc17fb7c17


In [5]:
# Attempt to build preload with new pce column

project.network.build_graphs()
project.activate()

g = project.network.graphs["c"]
g.set_skimming(["travel_time"])
g.set_graph("travel_time")
g.set_blocked_centroid_flows(False)
g.graph["capacity"] = 500
g.graph["travel_time"] = g.graph["distance"] / 50

def hr_to_sec(e):
    return int(e * 60 * 60)

start = hr_to_sec(7)
end = hr_to_sec(8)
inclusion_cond = "start"

/home/ubuntu/git/aequilibrae/aequilibrae/project/network/network.py:343: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = pd.read_sql(sql, conn).fillna(value=np.nan)


In [6]:
project.network.links

In [7]:
# Get trip_id based on specified inclusion condition
with read_and_close(database_connection("transit")) as conn:
    links = pd.read_sql(build_pt_preload_sql(start, end, inclusion_cond), conn)

preload = pd.merge(g.graph, links, on=["link_id", "direction"], how="left")
preload["preload"] = preload["preload"].fillna(0)
preload = preload.sort_values(by="__supernet_id__")["preload"].to_numpy()

In [8]:
preload_2 = project.network.build_pt_preload(g, hr_to_sec(start), hr_to_sec(end), inclusion_cond="start")

In [11]:
print(preload.sum(), preload_2.sum())
print(len(preload), len(preload_2))


12484.0

In [10]:
# %%
# project.close()